### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE68606'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking gene expression data availability
is_gene_available = True

# Identifying trait_row, age_row, and gender_row
for key, values in sample_characteristics.items():
    if any("Adrenal Cortical Adenoma" in element for element in values):
        trait_row = key
    if all(f"age" in element for element in values if element != '--'):
        age_row = key
    if any("Sex" in element for element in values):
        gender_row = key

# Define data conversion functions
def convert_trait(value):
    try:
        val = value.split(":")[-1].strip()
        if val in ["Adrenal Cortical Adenoma"]:
            return 1
        else:
            return 0
    except:
        return None

def convert_age(value):
    try:
        val = int(value.split(":")[-1].strip())
        return val
    except:
        return None

def convert_gender(value):
    try:
        val = value.split(":")[-1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE68606', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE68606.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
